# 入门

本笔记本演示了LangChain如何考虑内存。

内存涉及在用户与语言模型的交互过程中始终保持状态概念。用户与语言模型的交互被捕获在ChatMessages的概念中，因此这归结为从一系列聊天消息中摄取、捕获、转换和提取知识。有许多不同的方法来实现这一点，每种方法都作为其自己的内存类型存在。

通常，对于每种类型的内存，都有两种使用内存的方式。这些是独立的函数，从一系列消息中提取信息，然后是您可以在这种内存链中使用这种类型的方法。

内存可以返回多个信息片段（例如，最近的N个消息和所有先前消息的摘要）。返回的信息可以是字符串或消息列表。

在本笔记本中，我们将介绍最简单的存储器形式："缓冲器"存储器，它只涉及保留所有先前消息的缓冲区。我们将展示如何在此处使用模块化实用程序函数，然后展示它如何在链中使用（返回字符串以及消息列表）。

## 聊天消息历史
支撑大多数（如果不是所有）存储器模块的核心实用类之一是`ChatMessageHistory`类。这是一个超轻量级包装器，它公开了保存人类消息、AI消息，然后获取它们所有的方便方法。

如果您在链外管理内存，您可能希望直接使用此类。


In [2]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [3]:
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False)]

## ConversationBufferMemory

现在我们展示如何在链中使用这个简单的概念。我们首先展示`ConversationBufferMemory`，它只是一个围绕ChatMessageHistory的包装器，提取变量中的消息。

我们可以首先将其提取为字符串。

In [4]:
from langchain.memory import ConversationBufferMemory

In [5]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [7]:
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: whats up?'}

我们也可以将历史记录作为消息列表获取

In [8]:
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [9]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!', additional_kwargs={}, example=False),
  AIMessage(content='whats up?', additional_kwargs={}, example=False)]}

## 在链中使用
最后，让我们看一下如何在链中使用它（设置`verbose = True`，以便我们可以查看提示）。

In [10]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain


llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [11]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [12]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

In [13]:
conversation.predict(input="Tell me about yourself.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: Tell me about yourself.
AI:

> Finished chain.


" Sure! I'm an AI created to help people with their everyday tasks. I'm programmed to understand natural language and provide helpful information. I'm also constantly learning and updating my knowledge base so I can provide more accurate and helpful answers."

## 保存消息历史

您经常必须保存消息，然后重新加载以再次使用它们。这可以通过首先将消息转换为普通的Python字典，保存这些字典（如json或其他），然后加载这些字典来轻松完成。下面是一个这样做的例子。

In [14]:
import json

from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [15]:
dicts = messages_to_dict(history.messages)

In [16]:
dicts

[{'type': 'human',
  'data': {'content': 'hi!', 'additional_kwargs': {}, 'example': False}},
 {'type': 'ai',
  'data': {'content': 'whats up?', 'additional_kwargs': {}, 'example': False}}]

In [17]:
new_messages = messages_from_dict(dicts)

In [18]:
new_messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False)]

这就是入门所需的全部！有许多不同类型的内存，请查看我们的示例以查看它们全部